In [2]:
# Import necessary libraries
using CSV, DataFrames, ScikitLearn, Clustering, GaussianMixtures, Distances, Statistics

In [3]:
# LOAD DATASET AS DATAFRAME
df = CSV.File(read("01_hdp.csv")) |> DataFrame;

In [4]:
@sk_import preprocessing: LabelEncoder
@sk_import preprocessing: MinMaxScaler
le = LabelEncoder()
scaler = MinMaxScaler()

PyObject MinMaxScaler()

In [5]:
# ENCODE CATEGORICAL ATTRIBUTES

df[!, :FamilyHx] = le.fit_transform(df[!, :FamilyHx])
df[!, :SmokingHx] = le.fit_transform(df[!, :SmokingHx])
df[!, :Sex] = le.fit_transform(df[!, :Sex])
df[!, :CancerStage] = le.fit_transform(df[!, :CancerStage])
df[!, :School] = le.fit_transform(df[!, :School]);

In [6]:
# Convert df to an Array, and scale each column to a 0-1 range

X = convert(Array, df)


In [7]:
# Pairwise to get square distance matrix
dist = Euclidean()
R = pairwise(dist, X', X');

In [8]:
# Functions for cluster interpretations- based on counts and silhouettes
function count_interp(c)
    num = size(c)
    num = num[1]
    max = findmax(c)
    maxIndex = max[2]
    maxNum = max[1]
    min = findmin(c)
    minIndex = min[2]
    minNum = min[1]
    println("Number of clusters: $num")
    println("Largest group: $maxNum in cluster $maxIndex")
    println("Smallest group: $minNum in cluster $minIndex")
end

function sil_interp(sil)
    silMean = mean(sil)
    println("Silhouette mean: $silMean")
end

sil_interp (generic function with 1 method)

In [9]:
# cluster variable- 30 determined to be optimal
clusters = 30

30

In [10]:
# K means processing
meansResult = @time kmeans(R, clusters)
cost = meansResult.totalcost
println("Total cost: $cost\n")

sil_interp(silhouettes(meansResult, R))
println()
count_interp(counts(meansResult))

costs = meansResult.costs
println()
for i = 1:3
    maxCost = findmax(costs)
    index = maxCost[2]
    println("Outlier found at row $index")
    costs[index] = 0
end

 29.378882 seconds (7.94 M allocations: 386.162 MiB, 0.25% gc time)
Total cost: 2.227162873700039e6

Silhouette mean: 0.1473552866752826

Number of clusters: 30
Largest group: 868 in cluster 27
Smallest group: 114 in cluster 2

Outlier found at row 8459
Outlier found at row 8465
Outlier found at row 6605


In [11]:
# K medoids processing
medoidsResult = @time kmedoids(R, clusters)

cost = medoidsResult.totalcost
println("Total cost: $cost\n")

sil_interp(silhouettes(medoidsResult, R))
println()
count_interp(counts(medoidsResult))

costs = medoidsResult.costs
println()
for i = 1:3
    maxCost = findmax(costs)
    index = maxCost[2]
    println("Outlier found at row $index")
    costs[index] = 0
end


  0.548988 seconds (543.57 k allocations: 124.913 MiB)
Total cost: 9135.343782064108

Silhouette mean: 0.08379319843803962

Number of clusters: 30
Largest group: 783 in cluster 21
Smallest group: 63 in cluster 16

Outlier found at row 523
Outlier found at row 3081
Outlier found at row 8470


In [12]:
# Affinity Propogation processing
affPropResult = @time affinityprop(R; damp=:0.5)

println()
sil = silhouettes(affPropResult, R)
sil_interp(sil)
println()
count_interp(counts(affPropResult))

println()
# Find outliers with lowest silhouette scores
for i = 1:3
    minSil = findmin(sil)
    row = minSil[2]
    println("Outlier found at row $row")
    sil[row] = 0
end

 51.673121 seconds (289.79 k allocations: 2.193 GiB, 0.65% gc time)

Silhouette mean: 0.023211323699150785

Number of clusters: 69
Largest group: 586 in cluster 56
Smallest group: 40 in cluster 3

Outlier found at row 296
Outlier found at row 271
Outlier found at row 63


In [13]:
# Hierarchical cluster processing
hclu = @time hclust(R; linkage=:single, branchorder=:optimal)

hclustResult = cutree(hclu; k=100)

println()
sil = silhouettes(hclustResult, R)
sil_interp(sil)
println()
count_interp(counts(hclustResult))

println()
# Find outliers with lowest silhouette scores
for i = 1:3
    minSil = findmin(sil)
    row = minSil[2]
    println("Outlier found at row $row")
    sil[row] = 0
end

  5.274509 seconds (1.67 M allocations: 644.482 MiB, 12.48% gc time)

Silhouette mean: 0.11834529196605398

Number of clusters: 100
Largest group: 1218 in cluster 3
Smallest group: 1 in cluster 24

Outlier found at row 8157
Outlier found at row 6300
Outlier found at row 4207


In [14]:
gmmResult = @time GMM(R);

  1.607144 seconds (3.26 M allocations: 164.979 MiB)


In [15]:
print("Average log likelihood: ")
println(avll(gmmResult, R))
h = gmmposterior(gmmResult, R)
print("Mean of posterior probability: ")
println(mean(h[2]))

print(gmmResult)

Average log likelihood: -0.3378472841263621
Mean of posterior probability: -2880.148097177237


GMM{Float64,Array{Float64,2}}(1, 8525, [1.0], [2.0301070666253676 2.072916881456529 1.9869450623698015 2.1109074727336212 2.0422294378518506 2.0135265151794495 2.055965117385321 2.067942765918361 2.0876242614886507 2.0063410241418187 2.0915749182907977 1.9784114059864086 1.9148550012668801 2.05428690945747 2.209560093894366 2.0577527835453218 2.2342256172234722 2.121990092193814 2.208768353664041 1.9484149608891996 1.9846625019550164 2.243496750455624 2.00621696122036 2.179688879997868 2.1468967755411343 2.274186809464391 1.9528926740759174 2.0125464569877356 2.150167412236594 1.8870581729456648 1.8775547036242768 1.9977880962990047 2.1173181494677418 1.9408621372287753 1.9217636624632246 2.013785079661396 2.15290015107435 2.227990908084952 1.8569839786220723 1.9069564581735392 1.896789904341871 1.9791112807641085 2.0757814200134925 1.9515101593342996 2.079051355554592 1.9697766550181124 2.0117357558846685 2.0457540816773148 1.9701324950266408 2.1649331348484475 2.130231483104527 1.949

 1.882686092284916 2.0217478730452947 1.8537979467607282 1.7681283547619968 2.1578071707963122 1.8950275576587148 2.2035272093470613 1.8271725326124746 1.734642013594071 1.7944589409982554 1.911157644624758 1.9010157687408002 2.022105689622697 1.8348260757559782 2.038348779909053 1.8056770676070897 1.904467576330574 2.01310100158109 1.877589391367719 2.292106358680866 1.9172614404944084 1.7358687534043422 1.9365281527248992 2.1772119711045543 1.8449493816266538 1.9999713583605196 2.1204379153445925 2.224150022624872 2.2583386080080015 1.9294472959807794 2.0699667565445807 2.0177690461264164 2.042190735615084 2.0644804134875976 1.938790161516944 2.044299606112888 1.8786650757511918 2.104762365281135 1.9834358480500414 1.9726447783402963 2.1476471778395627 1.9211201438545058 2.0223641863197552 2.0138889440612298 2.005175311604104 2.0665781947521404 2.000767319821741 1.7330339896154021 1.768494812705823 1.8483994061645004 2.1799747332836126 1.8184394383667741 1.823513298834616 1.970496283

1.9207650784439987 2.0303913594962975 1.998052575406597 2.150874127421008 1.9366833600554276 2.1225562637654045 2.1332919098305867 1.881588091302714 1.9153452430133393 2.1729842305135127 2.075213381030819 2.058877635911497 1.979339050116894 1.9642109146922393 1.8717650469583311 1.8935027328834835 2.1907365849294487 1.9181806164347854 1.9391875465316373 2.163791645168464 1.9157284565816421 1.9335193866242872 1.7616103023767693 2.051137663281878 2.187788083778226 2.042361581349941 1.9062638805900243 1.8929896622520803 1.8884165348232036 1.9036540749257993 2.0122661028021303 2.0885443178283283 1.8319109012465007 2.0344762549785194 2.0426566355982096 1.8239669083511985 1.713682157600206 2.0379432420359986 1.8697391635249743 1.9074663066907205 2.0472810932387318 1.7764660138937758 1.8493995435736794 1.9763402977326052 1.8973825945930587 1.9323371299061987 2.0987889062121114 1.9224705936957527 1.84998479046652 1.7746737376271713 1.9373999954555938 2.252469667841014 2.2354713585391774 2.08368

 1.9729835470815689 1.855258582697306 1.831228523721538 1.8180922332432758 1.8617119960543922 2.1445551482256224 1.8237415835210746 1.8468384199861598 1.8515521185392108 1.833047378605134 2.0384613566653957 1.8404081464654738 2.1609864457902477 1.9585294906250288 1.9526989886145107 2.099840183049838 1.8625548315822196 2.092856211539318 2.0040812493931477 2.1462440133430487 2.0099066350661783 1.9017923209022667 2.11017588431666 1.7463730110155835 1.9263315376701027 1.9768256712518957 1.9220523545525225 1.9701167982744645 1.9129551482264024 1.8007641333762916 1.7745607349433785 1.7681355840756314 2.1113768733895286 1.9083239173411433 1.8463430605975584 2.2402503267243263 1.8746919975963054 1.8176369565077852 1.9764647207927457 1.8387653908081905 2.1605807278448004 2.068740015591614 1.9072412816386108 1.9030353170551155 1.8647418563650198 1.8097144372379534 2.0441841518519266 1.8705352498637315 1.7956432062672534 1.760621721497561 1.9146572536931254 1.7565933562698774 1.8408043552985478 1

...

 0.12089995795631925 0.12930516124983504 0.12560391883826658 0.12893541154819407 0.11002571229075303 0.12877335469650572 0.08570021051896436 0.11474931349221962 0.13054233494081618 0.12966501696974247 0.11959437210547304 0.11399248821149421 0.11533084962907413 0.09741824881846615 0.09184732858724727 0.11830192304126583 0.11598347141281373 0.13025287528413473 0.10828873743010221 0.12518993918030732 0.12840847817057877 0.11996739283516628 0.1193938086732339 0.13659619263857797 0.10036146909775903 0.09219762524371071 0.1155060490094888 0.08620226848712347 0.1290063895139094 0.13927903121321808 0.11752596932571775 0.11698619031982774 0.10201602332236616 0.12797719717052036 0.12218435913875253 0.12019551159681616 0.13985564347895182 0.09575083323984432 0.07843527314687626 0.10523464165706806 0.13328248812027982 0.12047447812814412 0.12681539330433989 0.09544445818232357 0.1255104829925222 0.10977938934730155 0.12294242791972199 0.13080784993471342 0.08529546234349972 0.1268073299564818 0.11

0.08183969160209013 0.09596417264565996 0.09395447880062484 0.11237897875095028 0.08819859035068735 0.10771979514954325 0.10100548211460206 0.10836431995921492 0.0933162301317009 0.10509252666713793 0.10436524389137627 0.1070662349250664 0.09624305971070052 0.09809794439226657 0.09218473730407584 0.10509186263659714 0.09029629910092087 0.09792257894987534 0.08832594796481737 0.11115320787490553 0.12152245082357432 0.13344666103809585 0.09850360207036193 0.12445399935072393 0.08399639008663602 0.14531699411504911 0.1121465730699574 0.1428216889386083 0.10169371957269245 0.12472197404085729 0.10229104638554304 0.09298069752286235 0.11492653011008057 0.09561933722571743 0.11503386846763328 0.08729513289250192 0.0928866273208055 0.1214280683034457 0.14373283423874297 0.1357173941963611 0.1144774318290652 0.123663173474422 0.13002935629438372 0.15164778486442748 0.109140324619011 0.12770886608181845 0.09524631439163952 0.12552242844362513 0.10497348784738432 0.1080342394996974 0.09096790398

0.097098531581386 0.1253927370002307 0.13471370033400573 0.09165211368360554 0.11636733496298914 0.09284193115049562 0.12474633777385916 0.11405277463900573 0.14503160870883305 0.12771641841030207 0.10149877519025678 0.10915568646361128 0.122775395035344 0.10749630672333865 0.11783499729963001 0.1269877605223052 0.1284535215192799 0.12565513911365925 0.11367514474486806 0.0935245356741667 0.13241301889205803 0.10961277087994271 0.11270171629876438 0.10221091038178605 0.0919946450522358 0.10637181634154136 0.10591983749907749 0.12772433019262433 0.11958353692440578 0.0971965354229527 0.12316461025376592 0.12330399635012092 0.10821525281298067 0.11275413301569584 0.1036276588031663 0.12749411868461316 0.13750023668697026 0.11551775682206714 0.1286576917971407 0.12135332528154896 0.1086468930729006 0.11351191068022015 0.12411443097284591 0.12944435830708506 0.11818706615198649 0.11693656780683438 0.11961831673176775 0.12155626841666009 0.13329309524163777 0.11208296936130992 0.12918992379

 0.1307656613622749 0.11142552323121326 0.12482292841193154 0.12226834860043127 0.11944286564579217 0.12076328369086178 0.09549451596134295 0.12699250029215675 0.0893203513900788 0.13696331097529968 0.12326079991561277 0.10253047881256748 0.11497136895676277 0.1218768634451592 0.12283164273852674 0.12386114681949009 0.12776899996215232 0.1051257565258832 0.10878892267294801 0.12151117599106405 0.13071852865189892 0.11438205024528925 0.13849987157139362 0.11516712376214086 0.1188158066113774 0.13991105759988223 0.13137658053596982 0.09219076288984218 0.1295130791458626 0.09433402610744372 0.11731939485334127 0.12645159617900129 0.12484282512036397 0.14423409921670555 0.12177946621527785 0.12153084225790078 0.12524921980908796 0.1176178873230495 0.11457653454738988 0.0943857492592022 0.1241741544746385 0.11541347109496632 0.12078042312606921 0.09629098896275852 0.09158313348191238 0.08864360122253953 0.1161943949149238 0.12297270781222501 0.09920851088368161 0.1033578048440734 0.10261990

History[4/12/2021 11:00:34 PM: Initlialized single Gaussian d=8525 kind=diag with 8525 data points
], 8525)

In [16]:
# For finding lowest probabilities (outliers)

for i = 1:3
    x = h[2]
    minProb = findmin(x)
    index = minProb[2]
    row = index[1]
    println("Outlier found at row $row")
    x[index] = 1
end

Outlier found at row 8051
Outlier found at row 8459
Outlier found at row 6605
